In [52]:
import numpy as np
import quaternion

dir = "/home/fabianbu/data/"


class Frame:
    def setTransform(self):
        self.T = np.eye(4, 4, dtype=np.float64)
        self.T[0:3,0:3] = quaternion.as_rotation_matrix(self.Q)
        self.T[0:3,3] = self.t
        
    def __init__(self,Q, t, time):
        self.Q = Q
        self.t = t
        self.time = time
        self.setTransform()

def load(ft):
    frames = []
    with open(ft) as f:
        lines = f.readlines()
        for l in lines:
            l = l.removesuffix('\n')
            l = l.split(' ')
            q = quaternion.quaternion(float(l[7]), float(l[4]),float(l[5]),float(l[6]))
            t = np.array([float(l[1]), float(l[2]), float(l[3])])
            time = float(l[0])
            f = Frame(q, t, time)
            frames.append(f)
    return frames

orb = load(f"{dir}traj.txt")
orb = load(f"/tmp/trajkf.txt")
gt = load(f"{dir}gnss_pose.tum")


i1 = 0
i2 = 15

print(orb[i1])
print(gt[i2])

i1 = 0
i2 = 0
while gt[i2].time < orb[i1].time:
    i2+=1
    

if i2 > 0 and abs(gt[i2].time - orb[i1].time) > abs(gt[i2-1].time - orb[i1].time):
    i2-=1

cb = [0, -0.50000011920928955,  0.86602526903152466,    1.003000020980835,
                  -1,                    0,                    0, 0.059999998658895493,
                   0, -0.86602526903152466, -0.50000011920928955,  0.73598074913024902,
                   0,                    0,                    0,                    1]
cb = np.array(cb).reshape((4,4))
ci =  [0.0079984068870544434,  0.0019884109497070312,    0.99996602535247803,  0.0020000000949949026,
  -0.99996602535247803,  0.0020043849945068359,  0.0079943835735321045,   0.023000000044703484,
-0.0019884109497070312,   -0.99999594688415527,  0.0020043849945068359,                     -0,
                     0,                      0,                      0,                      1]
ci = np.array(ci).reshape((4,4))
 
T1 = orb[i1]
T2 = gt[i2]



bc = np.linalg.inv(cb)
ic  = np.linalg.inv(ci)

T0 = np.linalg.inv(orb[0].T)

with open(dir+"out.txt", "w+") as f:
    lines = []
    for P in orb:
        s = ""
        P.T = gt[i2].T @ (cb @ (T0 @ P.T) @ bc)
        q = quaternion.from_rotation_matrix(P.T[0:3,0:3])
        s += f"{P.time} {P.T[0,3]} {P.T[1,3]} {P.T[2,3]} {q.x} {q.y} {q.z} {q.w}\n"
        lines.append(s)
    f.writelines(lines)
    
    
print((gt[-1].T - gt[0].T)[0:3,3])
print((orb[-1].T - orb[0].T)[0:3,3])

[[-1.11022302e-16 -1.11022302e-16  1.17961196e-16  0.00000000e+00]
 [-1.11022302e-16  2.22044605e-16  8.32667268e-17  0.00000000e+00]
 [-2.25514052e-17  1.38777878e-17  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
[47.05894322 24.99332085 -2.055     ]
[43.30592098 17.18754197 -3.73430899]
